In [1]:
from pathlib import Path
import numpy as np
from scipy.ndimage import label, distance_transform_edt
from skimage import io, img_as_float, filters, morphology
from skimage.feature import peak_local_max
from skimage.measure import regionprops, regionprops_table
from skimage.morphology import watershed
import pandas as pd
import glob
import skimage.measure as measure

import matplotlib.pyplot as plt

In [18]:
import pandas as pd

In [2]:
folder = r"G:\My Drive\Images\FL"
images = glob.glob(folder+'\*.tiff')

In [9]:
image = io.imread(r"G:\My Drive\Images\FL\009370z_025537x_006035y_093706F.tiff")

In [20]:
close_size=9
open_size=5
watershed_footprint = (13,13)
# Make sure types are the same
input_image = img_as_float(image)

# Filter Image
filtered_image = filters.median(input_image, behavior='ndimage')

# Edge Detection
edge_sobel = filters.sobel(filtered_image)

# Threshold
thresh = filters.threshold_otsu(edge_sobel)
binary_otsu = edge_sobel > thresh

# Binary Morphology Operations
structure_element = morphology.disk(close_size)
closed_image = morphology.binary_closing(binary_otsu, structure_element)
structure_element = morphology.disk(open_size)
opened_image = morphology.binary_opening(closed_image, structure_element)

# Watershed
distance = distance_transform_edt(opened_image)
local_maxi = peak_local_max(distance, indices=False, footprint=np.ones(watershed_footprint), labels=opened_image)
markers = label(local_maxi)[0]
labels = watershed(-distance, markers, mask=opened_image)

plt.imshow(labels)

regions = regionprops_table(labels, intensity_image=image, properties=('label', 'centroid', 'max_intensity', 'mean_intensity'))


df = pd.DataFrame(regions)

In [24]:
df.sort_values(by=['mean_intensity','max_intensity'], ascending=False)

,label,centroid-0,centroid-1,max_intensity,mean_intensity
3,4,269,1228,1371.0,718.079936
12,13,643,1237,1616.0,717.386585
13,14,689,1288,1344.0,666.810046
1,2,223,1006,1245.0,622.248499
14,15,715,1282,1317.0,609.839056
5,6,288,1174,1211.0,566.400438
2,3,235,1172,836.0,539.990741
4,5,284,1297,971.0,528.820823
6,7,368,1279,1124.0,512.935104
10,11,506,623,908.0,464.919649
